<a href="https://colab.research.google.com/github/mphirke/video2bvh2.0/blob/master/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Build Openpose on Colab (takes 15 minutes).
Reference : https://colab.research.google.com/github/tugstugi/dl-colab-notebooks/blob/master/notebooks/OpenPose.ipynb

In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

Selecting previously unselected package libgflags2.2.
(Reading database ... 144487 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

Remove existing directories (if they exist) and create required directories.

If you want to rerun, the best way would be to run after from here.

In [25]:
!rm -rf generated_bvh
!rm -rf generated_csv
!rm -rf generated_images
!rm -rf generated_jsons
!rm -rf uploaded_videos

In [26]:
!mkdir uploaded_videos
!mkdir generated_images
!mkdir generated_jsons

Upload video.

In [27]:
%cd /content/uploaded_videos
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


/content/uploaded_videos


Saving video(2).mp4 to video(2).mp4
User uploaded file "video(2).mp4" with length 8369334 bytes


In [28]:
!pip install natsort

Break down video to images. 
fps can be changed by changing the `fps` variable.

In [29]:
fps = 24

%matplotlib inline
import cv2
import matplotlib
import os

%cd /content/

for video in os.listdir('uploaded_videos'):
    print(video)
    vidcap = cv2.VideoCapture('uploaded_videos/'+video)
    success,image = vidcap.read()
    count = 0

    while True:
        try:
            vidcap.set(cv2.CAP_PROP_POS_MSEC,round((count*1000/fps)))    # added this line 
            success,image = vidcap.read()
            cv2.imwrite('generated_images/'+os.path.splitext(video)[0]+'_'+"frame%d.jpg" % count, image)     # save frame as JPEG file      
            count += 1
        except:
            break

/content
video(2).mp4


Run openpose over the images.

In [30]:
%cd /content/
! cd openpose && ./build/examples/openpose/openpose.bin --render_pose 0 --image_dir /content/generated_images --display 0 --write_json /content/generated_jsons --number_people_max 1

/content
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 118.113116 seconds.


Downloa the required checkpoints for hmr2.0

In [31]:
if 'logs' not in os.listdir():
    !mkdir logs
    !mkdir logs/lsp
    %cd logs/lsp
    !wget https://github.com/russoale/hmr2.0/releases/download/1.0/trained_lsp.zip
    !unzip trained_lsp

In [32]:
%cd /content/
!git clone https://github.com/russoale/hmr2.0.git
%cd hmr2.0/src
!pip install trimesh

/content
fatal: destination path 'hmr2.0' already exists and is not an empty directory.
/content/hmr2.0/src


In [39]:
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
import trimesh
import natsort

from os.path import join, abspath
from os import mkdir
from IPython.display import display, HTML
from glob import glob
sys.path.append(abspath('..'))

from main.config import Config
from main.model import Model
from main.dataset import Dataset
from main.smpl import Smpl
from main.local import LocalConfig

from notebooks.vis_util import draw_2d_on_image, show_2d_pose, show_3d_pose, preprocess_image, resize_img, visualize

Functions for preprocessing.
Reference : [openpose.py from akanazawa's hmr](https://github.com/akanazawa/hmr/blob/537f1b2c91beb71cb61a388bf8f6504516b875a8/src/util/openpose.py)

In [40]:
import numpy as np

def read_json(json_path):
    with open(json_path) as f:
        data = json.load(f)
    kps = []
    for people in data['people']:
        kp = np.array(people['pose_keypoints_2d']).reshape(-1, 3)
        kps.append(kp)
    return kps

def get_bbox(json_path, vis_thr=0.2):
    kps = read_json(json_path)
    # Pick the most confident detection.
    scores = [np.mean(kp[kp[:, 2] > vis_thr, 2]) for kp in kps]
    kp = kps[np.argmax(scores)]
    vis = kp[:, 2] > vis_thr
    vis_kp = kp[vis, :2]
    min_pt = np.min(vis_kp, axis=0)
    max_pt = np.max(vis_kp, axis=0)
    person_height = np.linalg.norm(max_pt - min_pt)
    if person_height == 0:
        print('bad!')
        import ipdb
        ipdb.set_trace()
    center = (min_pt + max_pt) / 2.
    scale = 150. / person_height

    return scale, center

def scale_and_crop(image, scale, center, img_size):
    image_scaled, scale_factors = resize_img(image, scale)
    # Swap so it's [x, y]
    scale_factors = [scale_factors[1], scale_factors[0]]
    center_scaled = np.round(center * scale_factors).astype(np.int)

    margin = int(img_size / 2)
    image_pad = np.pad(
        image_scaled, ((margin, ), (margin, ), (0, )), mode='edge')
    center_pad = center_scaled + margin
    # figure out starting point
    start_pt = center_pad - margin
    end_pt = center_pad + margin
    # crop:
    crop = image_pad[start_pt[1]:end_pt[1], start_pt[0]:end_pt[0], :]
    proc_param = {
        'scale': scale,
        'start': start_pt,
        'end': end_pt,
        'img_size': img_size
    }

    return crop, proc_param

In [41]:
import skimage.io as io
def preprocess_image_2(img_path, json_path=None):
    img = io.imread(img_path)
    if img.shape[2] == 4:
        img = img[:, :, :3]
    if json_path is None:
        if np.max(img.shape[:2]) != 224:
            print('Resizing so the max image size is %d..' % 224)
            scale = (float(config.img_size) / np.max(img.shape[:2]))
        else:
            scale = 1.
        center = np.round(np.array(img.shape[:2]) / 2).astype(int)
        # image center in (x,y)
        center = center[::-1]
    else:
        scale, center = get_bbox(json_path)

    crop, proc_param = scale_and_crop(img, scale, center,
                                               224)

    # Normalize image to [-1, 1]
    crop = 2 * ((crop / 255.) - 0.5)
    return crop, proc_param, img

Load frames and jsons.

In [42]:
from tqdm import tqdm

frames_path = '/content/generated_images/'
json_path = '/content/generated_jsons/'
frames = []
jsons = []
included_ext = [".png", ".jpg", ".jpeg"]
input_frames = [fn for fn in natsort.natsorted(os.listdir(frames_path)) if any(fn.endswith(ext) for ext in included_ext)]

for frame in tqdm(range(len(input_frames))): # Reference - https://stackoverflow.com/a/21096293/9155948
    frames.append(cv2.resize(cv2.imread(frames_path+input_frames[frame]), (224,224)))
    for openpose_json in natsort.natsorted(os.listdir(json_path)):
        if openpose_json.split('_')[-2] == os.path.splitext(input_frames[frame])[0].split('_')[-1]:
            jsons.append(openpose_json)


100%|██████████| 435/435 [00:16<00:00, 25.93it/s]


Preprocess frames (crop them around the detected human) and stored them into `cropped_frames`

In [43]:
import json
cropped_frames = []
proc_params = []
for frame_name, frame, openpose_json in tqdm(zip(input_frames,frames,jsons), total=len(input_frames)):
    cropped_image, proc_param, original_image = preprocess_image_2(frames_path+frame_name, json_path+openpose_json)
    cropped_frames.append(cropped_image)
    proc_params.append(proc_param)


 99%|█████████▉| 432/435 [00:18<00:00, 23.35it/s]

ValueError: ignored

Load configuration for hmr2.0.

In [44]:
class TrimeshConfig(LocalConfig):
        BATCH_SIZE = 1
        ENCODER_ONLY = True
        LOG_DIR = os.path.abspath('/content/logs/lsp')

        
config = TrimeshConfig()

In [45]:
# inizialize model 
model = Model()

Saving logs to /content/logs/lsp

Configurations:
BATCH_SIZE                     1
CUSTOM_REGRESSOR_IDX           {0: 'regressor_test.npy'}
CUSTOM_REGRESSOR_PATH          /content/hmr2.0/src/tests/files/regressors
DATASETS                       ['dataset']
DATA_DIR                       /content/hmr2.0/src/tests/files
DISCRIMINATOR_LEARNING_RATE    0.0001
DISCRIMINATOR_LOSS_WEIGHT      1
DISCRIMINATOR_WEIGHT_DECAY     0.0001
DS_KP2D                        {'lsp': 14, 'cocoplus': 19, 'custom': 21}
DS_KP3D                        {'lsp': 14, 'cocoplus': 14, 'custom': 16}
ENCODER_INPUT_SHAPE            (224, 224, 3)
ENCODER_ONLY                   True
EPOCHS                         55
GENERATOR_2D_LOSS_WEIGHT       60.0
GENERATOR_3D_LOSS_WEIGHT       60.0
GENERATOR_LEARNING_RATE        1e-05
GENERATOR_WEIGHT_DECAY         0.0001
ITERATIONS                     3
JOINT_TYPE                     cocoplus
LOG_DIR                        /content/logs/lsp
NUM_CAMERA_PARAMS              3
NUM_JOIN

Use Dene33's hmr fork 

In [46]:
%cd /content/
!git clone https://github.com/Dene33/hmr.git

/content
fatal: destination path 'hmr' already exists and is not an empty directory.


In [47]:
%mkdir generated_csv
%mkdir generated_csv/temp_generated_csv
%mkdir generated_bvh

Predict 3d keypoints using hmr2.0 and store them into csv.
Most of the following code was taken from Dene33's fork of hmr's file `demo.py`. Reference : https://github.com/Dene33/hmr/blob/master/demo.py

In [48]:
results = []
joints = []
vertices = []
img_2ds = []
cams = []
kp3ds = []

for i, image in enumerate(cropped_frames):
    result = model.detect(image)
    results.append(result)
    joint = np.squeeze(result['kp2d'].numpy())
    joints.append(joint)

    kp3d = np.squeeze(result['kp3d'].numpy())
    kp3ds.append(kp3d)

    img_2ds.append(draw_2d_on_image(image, joint))
    cams.append(np.squeeze(result['cam'].numpy())[:3])
    vertices.append(np.squeeze(result['vertices'].numpy()))
        
    joints_names = ['Ankle.R_x', 'Ankle.R_y', 'Ankle.R_z',
                   'Knee.R_x', 'Knee.R_y', 'Knee.R_z',
                   'Hip.R_x', 'Hip.R_y', 'Hip.R_z',
                   'Hip.L_x', 'Hip.L_y', 'Hip.L_z',
                   'Knee.L_x', 'Knee.L_y', 'Knee.L_z', 
                   'Ankle.L_x', 'Ankle.L_y', 'Ankle.L_z',
                   'Wrist.R_x', 'Wrist.R_y', 'Wrist.R_z', 
                   'Elbow.R_x', 'Elbow.R_y', 'Elbow.R_z', 
                   'Shoulder.R_x', 'Shoulder.R_y', 'Shoulder.R_z', 
                   'Shoulder.L_x', 'Shoulder.L_y', 'Shoulder.L_z',
                   'Elbow.L_x', 'Elbow.L_y', 'Elbow.L_z',
                   'Wrist.L_x', 'Wrist.L_y', 'Wrist.L_z', 
                   'Neck_x', 'Neck_y', 'Neck_z', 
                   'Head_x', 'Head_y', 'Head_z', 
                   'Nose_x', 'Nose_y', 'Nose_z', 
                   'Eye.L_x', 'Eye.L_y', 'Eye.L_z', 
                   'Eye.R_x', 'Eye.R_y', 'Eye.R_z', 
                   'Ear.L_x', 'Ear.L_y', 'Ear.L_z', 
                   'Ear.R_x', 'Ear.R_y', 'Ear.R_z']
    
    joints_export = pd.DataFrame(kp3d.reshape(1,57), columns=joints_names)
    joints_export.index.name = 'frame'
    
    joints_export.iloc[:, 1::3] = joints_export.iloc[:, 1::3]*-1
    joints_export.iloc[:, 2::3] = joints_export.iloc[:, 2::3]*-1

    hipCenter = joints_export.loc[:][['Hip.R_x', 'Hip.R_y', 'Hip.R_z',
                                      'Hip.L_x', 'Hip.L_y', 'Hip.L_z']]

    joints_export['hip.Center_x'] = hipCenter.iloc[0][::3].sum()/2
    joints_export['hip.Center_y'] = hipCenter.iloc[0][1::3].sum()/2
    joints_export['hip.Center_z'] = hipCenter.iloc[0][2::3].sum()/2
    
    joints_export.to_csv("generated_csv/temp_generated_csv/"+str(i)+".csv")


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [49]:
import glob
def join_csv(path = 'generated_csv/temp_generated_csv/'):          
  all_files = glob.glob(os.path.join(path, "*.csv"))
  all_files.sort(key=lambda x: int(x.split('/')[-1].split('.')[0]))
  print(all_files)
  df_from_each_file = (pd.read_csv(f) for f in all_files)
  concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)

  concatenated_df['frame'] = concatenated_df.index+1
  concatenated_df.to_csv("generated_csv/csv_joined.csv", index=False)

In [50]:
join_csv()

['generated_csv/temp_generated_csv/0.csv', 'generated_csv/temp_generated_csv/1.csv', 'generated_csv/temp_generated_csv/2.csv', 'generated_csv/temp_generated_csv/3.csv', 'generated_csv/temp_generated_csv/4.csv', 'generated_csv/temp_generated_csv/5.csv', 'generated_csv/temp_generated_csv/6.csv', 'generated_csv/temp_generated_csv/7.csv', 'generated_csv/temp_generated_csv/8.csv', 'generated_csv/temp_generated_csv/9.csv', 'generated_csv/temp_generated_csv/10.csv', 'generated_csv/temp_generated_csv/11.csv', 'generated_csv/temp_generated_csv/12.csv', 'generated_csv/temp_generated_csv/13.csv', 'generated_csv/temp_generated_csv/14.csv', 'generated_csv/temp_generated_csv/15.csv', 'generated_csv/temp_generated_csv/16.csv', 'generated_csv/temp_generated_csv/17.csv', 'generated_csv/temp_generated_csv/18.csv', 'generated_csv/temp_generated_csv/19.csv', 'generated_csv/temp_generated_csv/20.csv', 'generated_csv/temp_generated_csv/21.csv', 'generated_csv/temp_generated_csv/22.csv', 'generated_csv/temp_

Install blender

In [51]:
!apt-get install blender

Reading package lists... Done
Building dependency tree       
Reading state information... Done
blender is already the newest version (2.79.b+dfsg0-1ubuntu1.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


Change a few lines from Dene33's hmr file to match the new directories

In [52]:
!sed -i '16s#.*#directory = "generated_csv"#' hmr/csv_to_bvh.py
!sed -i '47s#.*#target_file = "generated_bvh/estimated_bvh.bvh"#' hmr/csv_to_bvh.py

In [53]:
!blender --background hmr/csv_to_bvh.blend -noaudio -P hmr/csv_to_bvh.py

Read blend: /content/hmr/csv_to_bvh.blend
[bpy.data.objects['Ankle.R'], bpy.data.objects['Knee.R'], bpy.data.objects['Hip.R'], bpy.data.objects['Hip.L'], bpy.data.objects['Knee.L'], bpy.data.objects['Ankle.L'], bpy.data.objects['Wrist.R'], bpy.data.objects['Elbow.R'], bpy.data.objects['Shoulder.R'], bpy.data.objects['Shoulder.L'], bpy.data.objects['Elbow.L'], bpy.data.objects['Wrist.L'], bpy.data.objects['Neck'], bpy.data.objects['Head'], bpy.data.objects['Nose'], bpy.data.objects['Eye.L'], bpy.data.objects['Eye.R'], bpy.data.objects['Ear.L'], bpy.data.objects['Ear.R'], bpy.data.objects['Hip.Center']]
BVH Exported: generated_bvh/estimated_bvh.bvh frames:436


Blender quit
src/tcmalloc.cc:283] Attempt to free invalid pointer 0x7fe81240e400 


In [54]:
files.download('generated_bvh/estimated_bvh.bvh')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**References:**

[Dene33's video to bvh](https://github.com/Dene33/video_to_bvh)

[hmr by akanazawa](https://github.com/akanazawa/hmr)

[Russoale's hmr2.0](https://github.com/russoale/hmr2.0)

[Openpose](
https://github.com/CMU-Perceptual-Computing-Lab/openpose)

[Openpose Colab notebook by tugstugi](https://colab.research.google.com/github/tugstugi/dl-colab-notebooks/blob/master/notebooks/OpenPose.ipynb)

[Dene33's fork of hmr](https://github.com/Dene33/hmr)

**hmr citation:**
```
@inProceedings{kanazawaHMR18,
  title={End-to-end Recovery of Human Shape and Pose},
  author = {Angjoo Kanazawa
  and Michael J. Black
  and David W. Jacobs
  and Jitendra Malik},
  booktitle={Computer Vision and Pattern Recognition (CVPR)},
  year={2018}
}
```
**Openpose citations:**
``````
@article{8765346,
  author = {Z. {Cao} and G. {Hidalgo Martinez} and T. {Simon} and S. {Wei} and Y. A. {Sheikh}},
  journal = {IEEE Transactions on Pattern Analysis and Machine Intelligence},
  title = {OpenPose: Realtime Multi-Person 2D Pose Estimation using Part Affinity Fields},
  year = {2019}
}

@inproceedings{simon2017hand,
  author = {Tomas Simon and Hanbyul Joo and Iain Matthews and Yaser Sheikh},
  booktitle = {CVPR},
  title = {Hand Keypoint Detection in Single Images using Multiview Bootstrapping},
  year = {2017}
}

@inproceedings{cao2017realtime,
  author = {Zhe Cao and Tomas Simon and Shih-En Wei and Yaser Sheikh},
  booktitle = {CVPR},
  title = {Realtime Multi-Person 2D Pose Estimation using Part Affinity Fields},
  year = {2017}
}

@inproceedings{wei2016cpm,
  author = {Shih-En Wei and Varun Ramakrishna and Takeo Kanade and Yaser Sheikh},
  booktitle = {CVPR},
  title = {Convolutional pose machines},
  year = {2016}
}
``````


